In [ ]:
!pip install nltk transformers torch scikit-learn flask

In [ ]:
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
class IntentDetectionAgent:
    def __init__(self):
        self.classifier = pipeline("text-classification", 
                                 model="bhadresh-savani/distilbert-base-uncased-emotion")
    def handle(self, message):
        return self.classifier(message)[0]['label']

In [ ]:
class FAQAgent:
    def __init__(self):
        self.faqs = {
            "What is your return policy?": "You can return products within 30 days.",
            "How to reset my password?": "Click 'Forgot Password' on the login screen.",
        }
        self.vectorizer = TfidfVectorizer()
        self.questions = list(self.faqs.keys())
        self.faq_vectors = self.vectorizer.fit_transform(self.questions)
    
    def handle(self, message):
        user_vec = self.vectorizer.transform([message])
        sims = cosine_similarity(user_vec, self.faq_vectors)
        return self.faqs[self.questions[sims.argmax()]]

In [ ]:
class SentimentAgent:
    def __init__(self):
        self.sentiment = pipeline("sentiment-analysis")
    def handle(self, message):
        return self.sentiment(message)[0]['label']

In [ ]:
class EscalationAgent:
    def handle(self, _, sentiment):
        return "Escalating to human agent!" if sentiment == "NEGATIVE" else "No escalation needed."

In [ ]:
class CustomerSupportSystem:
    def __init__(self):
        self.agents = {
            'intent': IntentDetectionAgent(),
            'faq': FAQAgent(),
            'sentiment': SentimentAgent(),
            'escalation': EscalationAgent()
        }
    
    def respond(self, message):
        intent = self.agents['intent'].handle(message)
        sentiment = self.agents['sentiment'].handle(message)
        response = self.agents['faq'].handle(message) if "question" in message.lower() else "Please clarify."
        return {
            "response": response,
            "sentiment": sentiment,
            "escalation": self.agents['escalation'].handle(message, sentiment)
        }

In [ ]:
# Try the system
bot = CustomerSupportSystem()
while True:
    msg = input("You: ")
    if msg.lower() in ["exit", "quit"]: break
    output = bot.respond(msg)
    print(f"\nResponse: {output['response']}\nSentiment: {output['sentiment']}\nAction: {output['escalation']}\n")